use simplified data 2

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

In [26]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [27]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME, encoding='ANSI')
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound, label_left_bound, data_right_bound=None, label_right_bound=None):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    if label_right_bound is None:
        label_right_bound = len(data.columns)
    if data_right_bound is None:
        data_right_bound = label_left_bound
    X = data.iloc[1:, list(range(data_left_bound, data_right_bound))]
    y = data.iloc[1:, list(range(label_left_bound, label_right_bound))]
    
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

def SplitNparr(original_arr: np.ndarray, train_portion: float)->tuple:
    data_len =len(original_arr)
    train_len = int(data_len * train_portion)
    indices = list(range(data_len))
    random.shuffle(indices)
    train_idx = indices[:train_len]
    validate_idx = indices[train_len:]
    training_arr  = original_arr[train_idx]
    validation_arr = original_arr[validate_idx]

    return (training_arr,validation_arr)


def Split1Df(data: pd.DataFrame,  train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - data: any data
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - data_train: data for the training set.
    - data_val: data for the validation set.

    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(data)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]


    data_train, data_val = data.loc[train_idx], data.loc[val_idx]
   
    return data_train, data_val

def SplitBothXy_Df(X: pd.DataFrame, y: pd.DataFrame, train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - X: Features 
    - y: Target variable
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - X_train: Features for the training set.
    - X_val: Features for the validation set.
    - y_train: Target variable for the training set.
    - y_val: Target variable for the validation set.
    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(X)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]


    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    return X_train, X_val, y_train, y_val

In [28]:
data_pd = ReadData("./simplified_data/simplified_data2.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).


In [29]:
first_medicine_idx = 113

# split data into X and y
# x= all symptoms, diagnosis, body status
# y= all medince
X,y = SplitXY(data_pd, data_left_bound=2, data_right_bound=first_medicine_idx, label_left_bound=first_medicine_idx)

# X is a df, split X into trainX and testX
#train_X, val_X, train_y, val_y = SplitBothXy_Df(X, y, 0.8) # split need bug fix!!



SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1   1   0   0
5    0   1   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    0   1   0   0   0   0   0   0   0   0
8    0   1   0   0  

In [30]:

X_np = X.values.astype('float64')
print("X transformed to np array")
print("type of X_np:", type(X_np))
print("shape of X_np:", X_np.shape)
num_col_x = X_np.shape[1]
print("number of col in x:",num_col_x)
print("len of y:", y.shape[1])



X transformed to np array
type of X_np: <class 'numpy.ndarray'>
shape of X_np: (796, 111)
number of col in x: 111
len of y: 102


In [31]:
# build and compile model

model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1792      
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,402
Trainable params: 2,402
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [33]:

#for i in range(y.shape[1]):
for i in range(2):
    chosen_col = y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(y))
    print("processing medicine:", chosen_col.name)
    chosen_y_np = chosen_col.values.astype('float64')
    model.fit(x=X_np, y=chosen_y_np,
          batch_size=10, epochs=10, shuffle=True, verbose=2)



processing medicine: 麻黃
Epoch 1/10
80/80 - 2s - loss: 0.4775 - accuracy: 0.8229 - 2s/epoch - 25ms/step
Epoch 2/10
80/80 - 0s - loss: 0.4161 - accuracy: 0.8342 - 210ms/epoch - 3ms/step
Epoch 3/10
80/80 - 0s - loss: 0.3656 - accuracy: 0.8392 - 213ms/epoch - 3ms/step
Epoch 4/10
80/80 - 0s - loss: 0.3199 - accuracy: 0.8543 - 209ms/epoch - 3ms/step
Epoch 5/10
80/80 - 0s - loss: 0.2494 - accuracy: 0.8869 - 212ms/epoch - 3ms/step
Epoch 6/10
80/80 - 0s - loss: 0.2053 - accuracy: 0.9083 - 217ms/epoch - 3ms/step
Epoch 7/10
80/80 - 0s - loss: 0.1774 - accuracy: 0.9384 - 212ms/epoch - 3ms/step
Epoch 8/10
80/80 - 0s - loss: 0.1464 - accuracy: 0.9485 - 222ms/epoch - 3ms/step
Epoch 9/10
80/80 - 0s - loss: 0.1094 - accuracy: 0.9598 - 222ms/epoch - 3ms/step
Epoch 10/10
80/80 - 0s - loss: 0.0886 - accuracy: 0.9686 - 214ms/epoch - 3ms/step
processing medicine: 桂枝
Epoch 1/10
80/80 - 0s - loss: 0.9430 - accuracy: 0.5704 - 216ms/epoch - 3ms/step
Epoch 2/10
80/80 - 0s - loss: 0.6394 - accuracy: 0.6357 - 218m